## Classification and dimensionality reduction

In [1]:
%load_ext autoreload
%autoreload 2

## Imports of packages

In [2]:
import numpy as np
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
import torch
from torch import nn
from torchvision import transforms
from torchvision.datasets import SVHN
from torchvision.models import resnet18
from PIL import Image

In [3]:
from triplets.transformers import FeatureExtractor

In [4]:
batch_size = 1
n_features = 512
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

Defining transformations of the dataset

In [5]:
preprocess = transforms.Compose([            
 transforms.Resize(256),                    
 transforms.CenterCrop(224),                
 transforms.ToTensor(),                     
 transforms.Normalize(                      
 mean=[0.485, 0.456, 0.406],                
 std=[0.229, 0.224, 0.225]                  
 )])

Loading dataset

In [7]:
dataset = SVHN(root = 'data/', download=True, split='train', transform=preprocess)

Using downloaded and verified file: data/train_32x32.mat


Loading pretrained resnet
#TODO: Describe the network and justify its use

In [8]:
model = resnet18(pretrained=True)
model.eval();

In [9]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size)

Defining extractor using custom class to extract features from last cnn pretrained layer.

In [10]:
extractor = FeatureExtractor(model=model, n_remove_layers=1, n_features=n_features, 
                             dataloader=dataloader, device=device)

In [1]:
X_train_codes, y_train_codes = extractor.extract_features()

Training kNN-classifier on top of cnn codes with cross-validated grid search

In [ ]:
params = {'n_neighbors': list(range(3, 10))}
classifier = KNeighborsClassifier()
clf = GridSearchCV(classifier, params, cv=10)
clf.fit(X_train_codes, y_train_codes)